In [ ]:
#This code can be used to reproduce the forecasts submitted to the M4 competition for the 4Theta method

#Authors: E. Spiliotis and V. Assimakopoulos (2017) / Forecasting & Strategy Unit - NTUA

#Method Description: Generalizing the Theta model for automatic forecasting
#Method Type: Statistical - Decomposition

library(forecast) #requires version 8.2

SeasonalityTest <- function(input, ppy){
  #Used for determining whether the time series is seasonal
  tcrit <- 1.645
  if (length(input)<3*ppy){
    test_seasonal <- FALSE
  }else{
    xacf <- acf(input, plot = FALSE)$acf[-1, 1, 1]
    clim <- tcrit/sqrt(length(input)) * sqrt(cumsum(c(1, 2 * xacf^2)))
    test_seasonal <- ( abs(xacf[ppy]) > clim[ppy] )
    
    if (is.na(test_seasonal)==TRUE){ test_seasonal <- FALSE }
  }
  
  return(test_seasonal)
}

Theta.fit <- function(input, fh, theta, curve, model, seasonality , plot=FALSE){
  #Used to fit a Theta model
  
  #Check if the inputs are valid
  if (theta<0){ theta <- 2  }
  if (fh<1){ fh <- 1  }
  #Estimate theta line weights
  outtest <- naive(input, h=fh)$mean
  if (theta==0){
    wses <- 0
  }else{
    wses <- (1/theta)
  }
  wlrl <- (1-wses)
  #Estimate seasonaly adjusted time series
  ppy <- frequency(input)
  if (seasonality=="N"){
    des_input <- input ; SIout <- rep(1, fh) ; SIin <- rep(1, length(input))
  }else if (seasonality=="A"){
    Dec <- decompose(input, type="additive")
    des_input <- input-Dec$seasonal 
    SIin <- Dec$seasonal
    SIout <- head(rep(Dec$seasonal[(length(Dec$seasonal)-ppy+1):length(Dec$seasonal)], fh), fh)
  }else{
    Dec <- decompose(input, type="multiplicative")
    des_input <- input/Dec$seasonal 
    SIin <- Dec$seasonal
    SIout <- head(rep(Dec$seasonal[(length(Dec$seasonal)-ppy+1):length(Dec$seasonal)], fh), fh)
  }
  
  #If negative values, force to linear model
  if (min(des_input)<=0){ curve <- "Lrl" ; model <- "A"  }
  #Estimate theta line zero
  observations <- length(des_input)
  xs <- c(1:observations)
  xf = xff <- c((observations+1):(observations+fh))
  dat=data.frame(des_input=des_input, xs=xs)
  newdf <- data.frame(xs = xff)
  
  if (curve=="Exp"){
    estimate <- lm(log(des_input)~xs)
    thetaline0In <- exp(predict(estimate))+input-input
    thetaline0Out <- exp(predict(estimate, newdf))+outtest-outtest
  }else{
    estimate <- lm(des_input ~ poly(xs, 1, raw=TRUE))
    thetaline0In <- predict(estimate)+des_input-des_input
    thetaline0Out <- predict(estimate, newdf)+outtest-outtest
  }
  
  #Estimete Theta line (theta)
  if (model=="A"){
    thetalineT <- theta*des_input+(1-theta)*thetaline0In
  }else if ((model=="M")&(all(thetaline0In>0)==T)&(all(thetaline0Out>0)==T)){
    thetalineT <- (des_input^theta)*(thetaline0In^(1-theta))
  }else{
    model<-"A"
    thetalineT <- theta*des_input+(1-theta)*thetaline0In
  }
  
  #forecasting TL2
  sesmodel <- ses(thetalineT, h=fh)
  thetaline2In <- sesmodel$fitted
  thetaline2Out <- sesmodel$mean
  
  #Theta forecasts
  if (model=="A"){
    forecastsIn <- as.numeric(thetaline2In*wses)+as.numeric(thetaline0In*wlrl)+des_input-des_input
    forecastsOut <- as.numeric(thetaline2Out*wses)+as.numeric(thetaline0Out*wlrl)+outtest-outtest
  }else if ((model=="M")&
            (all(thetaline2In>0)==T)&(all(thetaline2Out>0)==T)&
            (all(thetaline0In>0)==T)&(all(thetaline0Out>0)==T)){
    forecastsIn <- ((as.numeric(thetaline2In)^(1/theta))*(as.numeric(thetaline0In)^(1-(1/theta))))+des_input-des_input
    forecastsOut <- ((as.numeric(thetaline2Out)^(1/theta))*(as.numeric(thetaline0Out)^(1-(1/theta))))+outtest-outtest
  }else{
    model<-"A"
    thetalineT <- theta*des_input+(1-theta)*thetaline0In
    sesmodel <- ses(thetalineT,h=fh)
    thetaline2In <- sesmodel$fitted
    thetaline2Out <- sesmodel$mean
    forecastsIn <- as.numeric(thetaline2In*wses)+as.numeric(thetaline0In*wlrl)+des_input-des_input
    forecastsOut <- as.numeric(thetaline2Out*wses)+as.numeric(thetaline0Out*wlrl)+outtest-outtest
  }
  
  #Seasonal adjustments
  if (seasonality=="A"){
    forecastsIn <- forecastsIn+SIin
    forecastsOut <- forecastsOut+SIout
  }else{
    forecastsIn <- forecastsIn*SIin
    forecastsOut <- forecastsOut*SIout
  }
  
  #Zero forecasts become positive
  for (i in 1:length(forecastsOut)){
    if (forecastsOut[i]<0){ forecastsOut[i] <- 0 }
  }
  
  if (plot==TRUE){
    united <- cbind(input,forecastsOut)
    for (ik in 1:(observations+fh)){ united[ik,1] = sum(united[ik,2],united[ik,1], na.rm = TRUE) }
    plot(united[,1],col="black",type="l",main=paste("Model:",model,",Curve:",curve,",Theta:",theta),xlab="Time",ylab="Values",
         ylim=c(min(united[,1])*0.85,max(united[,1])*1.15))
    lines(forecastsIn, col="green") ; lines(forecastsOut, col="green")
    lines(thetaline2In, col="blue") ; lines(thetaline2Out, col="blue")
    lines(thetaline0In, col="red") ; lines(thetaline0Out, col="red")
  }
  
  output=list(fitted=forecastsIn,mean=forecastsOut,
              fitted0=thetaline0In,mean0=thetaline0Out,
              fitted2=thetaline2In,mean2=thetaline2Out,
              model=paste(seasonality,model,curve,c(round(theta,2))))
  
  return(output)
}

FourTheta<- function(input, fh){
  #Used to automatically select the best Theta model
  
  #Scale
  base <- mean(input) ; input <- input/base
  
  molist <- c("M","A") ; trlist <- c("Lrl","Exp") 
  
  #Check seasonality & Create list of models
  ppy <- frequency(input) ; ST <- F
  if (ppy>1){ ST <- SeasonalityTest(input, ppy) }
  if (ST==T){
    
    selist <- c("M","A")
    listnames <- c()
    for (i in 1:length(selist)){
      for (ii in 1:length(molist)){
        for (iii in 1:length(trlist)){
          listnames <- c(listnames,paste(selist[i], molist[ii], trlist[iii]))
        }
      }
    }
    
  }else{
    
    listnames <- c()
    for (ii in 1:length(molist)){
      for (iii in 1:length(trlist)){
        listnames <- c(listnames, paste("N", molist[ii], trlist[iii]))
      }
    }
    
  }
  
  modellist <- NULL
  for (i in 1:length(listnames)){
    modellist[length(modellist)+1] <- list(c(substr(listnames,1,1)[i], substr(listnames,3,3)[i],
                                             substr(listnames,5,7)[i]))
  }
  
  #Start validation
  errorsin <- c() ; models <- NULL
  
  #With this function determine opt theta per case
  optfun <- function(x, input, fh, curve, model, seasonality){
    mean(abs(Theta.fit(input=input, fh, theta=x, curve, model, seasonality , plot=FALSE)$fitted-input))
  }
  
  for (j in 1:length(listnames)){
    optTheta <- optimize(optfun, c(1:3), 
                         input=input, fh=fh, curve=modellist[[j]][3], model=modellist[[j]][2], 
                         seasonality=modellist[[j]][1])$minimum
    
    fortheta <- Theta.fit(input=input, fh=fh, theta=optTheta, curve=modellist[[j]][3], model=modellist[[j]][2], 
                          seasonality=modellist[[j]][1], plot=F)
    models[length(models)+1] <- list(fortheta)
    errorsin <- c(errorsin, mean(abs(input-fortheta$fitted)))
  }
  
  #Select model and export
  selected.model <- models[[which.min(errorsin)]]
  description <- selected.model$model
  output <- list(fitted=selected.model$fitted*base,mean=selected.model$mean*base,
                 description=description) 
  #Returns the fitted and forecasted values, as well as the model used (Type of seasonality, Type of Model, Type of Trend, Theta coef.)
  
  return(output)
  
}


In [ ]:
library(dplyr)
library(forecast) #Requires v8.2

library(readr)
smape_cal <- function(outsample, forecasts){
  #Used to estimate sMAPE
  outsample <- as.numeric(outsample) ; forecasts<-as.numeric(forecasts)
  smape <- (abs(outsample-forecasts)*200)/(abs(outsample)+abs(forecasts))
  return(smape)
}

mase_cal <- function(insample, outsample, forecasts){
  #Used to estimate MASE
  frq <- frequency(insample)
  forecastsNaiveSD <- rep(NA,frq)
  for (j in (frq+1):length(insample)){
    forecastsNaiveSD <- c(forecastsNaiveSD, insample[j-frq])
  }
  masep<-mean(abs(insample-forecastsNaiveSD),na.rm = TRUE)
  
  outsample <- as.numeric(outsample) ; forecasts <- as.numeric(forecasts)
  mase <- (abs(outsample-forecasts))/masep
 # print('-------------------------')
 # glimpse(mase)
  return(mase)
}

naive_seasonal <- function(input, fh){
  #Used to estimate Seasonal Naive
  frcy <- frequency(input)
  frcst <- naive(input, h=fh)$mean 
  if (frcy>1){ 
    frcst <- head(rep(as.numeric(tail(input,frcy)), fh), fh) + frcst - frcst
  }
  return(frcst)
}

Theta.classic <- function(input, fh){
  #Used to estimate Theta classic
  
  #Set parameters
  wses <- wlrl<-0.5 ; theta <- 2
  #Estimate theta line (0)
  observations <- length(input)
  xt <- c(1:observations)
  xf <- c((observations+1):(observations+fh))
  train <- data.frame(input=input, xt=xt)
  test <- data.frame(xt = xf)
  
  estimate <- lm(input ~ poly(xt, 1, raw=TRUE))
  thetaline0In <- as.numeric(predict(estimate))
  thetaline0Out <- as.numeric(predict(estimate,test))
  
  #Estimate theta line (2)
  thetalineT <- theta*input+(1-theta)*thetaline0In
  sesmodel <- ses(thetalineT, h=fh)
  thetaline2In <- sesmodel$fitted
  thetaline2Out <- sesmodel$mean
  
  #Theta forecasts
  forecastsIn <- (thetaline2In*wses)+(thetaline0In*wlrl)
  forecastsOut <- (thetaline2Out*wses)+(thetaline0Out*wlrl)
  
  #Zero forecasts become positive
  for (i in 1:length(forecastsOut)){
    if (forecastsOut[i]<0){ forecastsOut[i]<-0 }
  }
  
  output=list(fitted = forecastsIn, mean = forecastsOut,
              fitted0 = thetaline0In, mean0 = thetaline0Out,
              fitted2 = thetaline2In, mean2 = thetaline2Out)
  
  return(output)
}

SeasonalityTest <- function(input, ppy){
  #Used to determine whether a time series is seasonal
  tcrit <- 1.645
  if (length(input)<3*ppy){
    test_seasonal <- FALSE
  }else{
    xacf <- acf(input, plot = FALSE)$acf[-1, 1, 1]
    clim <- tcrit/sqrt(length(input)) * sqrt(cumsum(c(1, 2 * xacf^2)))
    test_seasonal <- ( abs(xacf[ppy]) > clim[ppy] )
    
    if (is.na(test_seasonal)==TRUE){ test_seasonal <- FALSE }
  }
  
  return(test_seasonal)
}

Benchmarks <- function(input, fh){
  #Used to estimate the statistical benchmarks of the M4 competition
  
  #Estimate seasonaly adjusted time series
  ppy <- frequency(input) ; ST <- F
  if (ppy>1){ ST <- SeasonalityTest(input,ppy) }
  if (ST==T){
    Dec <- decompose(input,type="multiplicative")
    des_input <- input/Dec$seasonal
    SIout <- head(rep(Dec$seasonal[(length(Dec$seasonal)-ppy+1):length(Dec$seasonal)], fh), fh)
  }else{
    des_input <- input ; SIout <- rep(1, fh)
  }
  
  f1 <- naive(input, h=fh)$mean #Naive
  f2 <- naive_seasonal(input, fh=fh) #Seasonal Naive
  f3 <- naive(des_input, h=fh)$mean*SIout #Naive2
  f4 <- ses(des_input, h=fh)$mean*SIout #Ses
  f5 <- holt(des_input, h=fh, damped=F)$mean*SIout #Holt
  f6 <- holt(des_input, h=fh, damped=T)$mean*SIout #Damped
  f7 <- Theta.classic(input=des_input, fh=fh)$mean*SIout #Theta
  f8 <- (f4+f5+f6)/3 #Comb
  
  return(list(f1,f2,f3,f4,f5,f6,f7,f8))
}

In [ ]:

#################################################################################

calcBenchmark <- function(path, fh,frq){


    df <- read_csv(path)
    df <-select(df,-V1)
    #dim(df)[1]



    #################################################################################
    #In this example let us produce forecasts for 100 randomly generated timeseries
    #fh <- 6 #The forecasting horizon examined
    #frq <- 1 #The frequency of the data
    data_train = data_test <- NULL #Train and test sample
    l<-dim(df)[1]
    #print(l)
    for (i in 1:l){
      data_all <-  as.vector(as.data.frame((df[i,])))
      data_all <-  data_all[!is.na(data_all)]
      #print(length(data_all))
      if (length(data_all>0)) {
          #glimpse(data_all)
          data_all <- as.double(data_all)
          data_train[length(data_train)+1] <- list(ts(head(data_all,length(data_all)-fh),frequency = frq))
          data_test[length(data_test)+1] <- list(tail(data_all,fh))
        }
      }




    Names_benchmarks <- c("Naive", "sNaive", "Naive2", "SES", "Holt", "Damped", "Theta", "Com")
    Total_smape=Total_mase <- array(NA,dim = c(length(Names_benchmarks), length(data_train)))
    print(dim(Total_smape))
    #Methods, Horizon, time-series
    for (i in 1:length(data_train)){

      insample <- data_train[[i]]
      outsample <- data_test[[i]]
      forecasts <- Benchmarks(input=insample, fh=fh)

      #sMAPE
      for (j in 1:length(Names_benchmarks)){
        Total_smape[j,i] <- mean(smape_cal(outsample, forecasts[[j]])) #j the # of the benchmark
      }
      #MASE
      for (j in 1:length(Names_benchmarks)){
        Total_mase[j,i] <- mean(mase_cal(insample, outsample, forecasts[[j]])) #j the # of the benchmark
      }
    }
    glimpse(Total_smape)  

    return(list(v1=Total_mase,v2=Total_smape))
}


In [151]:
library(gdata)
p=c('Daily','Weekly','Hourly','Monthly','Quaterly','Yearly')
fcs=c(14,13,48,18,8,1)
frqs=c(1,1,24,12,4,1)
size=100





test1=NULL
test2=NULL
for (x in 1:length(p)){
    #path=paste("../data/csv_5000_sample/", toString(p[x]),'-train-5000-sample.csv', sep = "")
    path=paste("../data/csv_",size,"_sample/", toString(p[x]),'-train-',size,'-sample.csv', sep = "")
    #path=paste("../data/csv_100_sample/", toString(p[x]),'-train-100-sample.csv', sep = "")

    test=calcBenchmark(path,fcs[x],frqs[x])
    test1[x]=test[1]
    test2[x]=test[2]

}

Names_benchmarks <- c("Naive", "sNaive", "Naive2", "SES", "Holt", "Damped", "Theta", "Com")

v1 = array(1, dim=c(8,length(p)*size))
v2 = array(1, dim=c(8,length(p)*size))
    glimpse(v1)

for (x in 1:length(p)){   
    v1[,((x-1)*size+1):(x*size)]=test1[[x]]
    v2[,((x-1)*size+1):(x*size)]=test2[[x]]
}
glimpse(v1)
glimpse(v2)

Total_mase=v2
Total_smape=v1
result=array(NA,dim = c(length(Names_benchmarks), 4))
print("########### sMAPE ###############")
for (i in 1:length(Names_benchmarks)){
  smape=round(mean(Total_smape[i,]), 3)
  mase=round(mean(Total_mase[i,]), 3)
  owa=round(((mean(Total_mase[i,])/mean(Total_mase[3,]))+(mean(Total_smape[i,])/mean(Total_smape[3,])))/2, 3)
  
  result[i,1]=Names_benchmarks[i]
  result[i,2]=smape
  result[i,3]=mase
  result[i,4]=owa  
  
  print(paste(Names_benchmarks[i], smape, mase, owa))
}
write.csv(result, file = paste("../results/R-Benchmark-",size,'-sample.csv'))

Parsed with column specification:
cols(
  .default = col_character(),
  V2 = col_double(),
  V3 = col_double(),
  V4 = col_double(),
  V5 = col_double(),
  V6 = col_double(),
  V7 = col_double(),
  V8 = col_double(),
  V9 = col_double(),
  V10 = col_double(),
  V11 = col_double(),
  V12 = col_double(),
  V13 = col_double(),
  V14 = col_double(),
  V15 = col_double(),
  V16 = col_double(),
  V17 = col_double(),
  V18 = col_double(),
  V19 = col_double(),
  V20 = col_double(),
  V21 = col_double()
  # ... with 4177 more columns
)
See spec(...) for full column specifications.


[1]   8 100
 num [1:8, 1:100] 0.701 0.701 0.701 0.701 0.738 ...


Parsed with column specification:
cols(
  .default = col_double(),
  V1 = col_character()
)
See spec(...) for full column specifications.


[1]   8 100
 num [1:8, 1:100] 9.12 9.12 9.12 9.12 15.83 ...


Parsed with column specification:
cols(
  .default = col_double(),
  V1 = col_character()
)
See spec(...) for full column specifications.


...

[1]   8 100
 num [1:8, 1:100] 6.51 6.26 6.94 6.8 6.51 ...


Parsed with column specification:
cols(
  .default = col_character(),
  V2 = col_double(),
  V3 = col_double(),
  V4 = col_double(),
  V5 = col_double(),
  V6 = col_double(),
  V7 = col_double(),
  V8 = col_double(),
  V9 = col_double(),
  V10 = col_double(),
  V11 = col_double(),
  V12 = col_double(),
  V13 = col_double(),
  V14 = col_double(),
  V15 = col_double(),
  V16 = col_double(),
  V17 = col_double(),
  V18 = col_double(),
  V19 = col_double(),
  V20 = col_double(),
  V21 = col_double()
  # ... with 203 more columns
)
See spec(...) for full column specifications.


[1]   8 100
 num [1:8, 1:100] 23.4 29 23.4 23.4 60.9 ...


Parsed with column specification:
cols(
  .default = col_character(),
  V2 = col_double(),
  V3 = col_double(),
  V4 = col_double(),
  V5 = col_double(),
  V6 = col_double(),
  V7 = col_double(),
  V8 = col_double(),
  V9 = col_double(),
  V10 = col_double(),
  V11 = col_double(),
  V12 = col_double(),
  V13 = col_double(),
  V14 = col_double(),
  V15 = col_double(),
  V16 = col_double(),
  V17 = col_double(),
  V18 = col_double(),
  V19 = col_double(),
  V20 = col_double(),
  V21 = col_double()
  # ... with 89 more columns
)
See spec(...) for full column specifications.


[1]   8 100
 num [1:8, 1:100] 99.2 99.2 99.2 99.2 90.1 ...
 num [1:8, 1:600] 1 1 1 1 1 1 1 1 1 1 ...
 num [1:8, 1:600] 0.307 0.307 0.307 0.307 0.324 ...
 num [1:8, 1:600] 0.701 0.701 0.701 0.701 0.738 ...
[1] "########### sMAPE ###############"
[1] "Naive 3.559 14.843 1.597"
[1] "sNaive 1.839 10.675 0.965"
[1] "Naive2 1.963 10.749 1"
[1] "SES 1.917 10.356 0.97"
[1] "Holt 2.622 12.652 1.256"
[1] "Damped 1.897 10.773 0.984"
[1] "Theta 1.885 10.336 0.961"
[1] "Com 2.058 11.03 1.037"


In [152]:
result

Naive,3.559,14.843,1.597
sNaive,1.839,10.675,0.965
Naive2,1.963,10.749,1
SES,1.917,10.356,0.97
Holt,2.622,12.652,1.256
Damped,1.897,10.773,0.984
Theta,1.885,10.336,0.961
Com,2.058,11.03,1.037
